# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [58]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import matplotlib.pyplot as plt
import numpy as np
import gmaps
import os

# Import API key
from api_keys import geoapify_key as g_key


In [59]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,yellowknife,62.4560,-114.3525,-20.71,65,100,4.63,CA,1670205320
1,1,ayan,56.4500,138.1667,-13.74,58,100,5.68,RU,1670205605
2,2,kruisfontein,-34.0033,24.7314,14.02,86,97,3.44,ZA,1670205606
3,3,saint-georges,12.0564,-61.7485,24.05,83,54,1.96,GD,1670205607
4,4,weston,26.1004,-80.3998,24.43,77,95,3.40,US,1670205606


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [60]:
%%capture --no-display

# Configure the map plot
mat_plot_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",  
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    legend = 'right')


# Display the map
mat_plot_map
     




:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [69]:
# Narrow down cities that fit criteria and drop any results with null values
Ideal_city_data_df = city_data_df[(city_data_df['Cloudiness'] == 0) & 
                                  (city_data_df['Max Temp'] < 27) & 
                                  (city_data_df['Max Temp'] > 21) & 
                                  (city_data_df['Wind Speed'] < 4.5)]

# Set Index
# Drop any rows with null values
# Display sample data

Ideal_city_data_df.dropna()
Ideal_city_data_df





,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,12,bitkine,11.9801,18.2138,21.64,15,0,3.00,TD,1670205611
36,36,dzilam gonzalez,21.2833,-88.9333,23.81,74,0,3.31,MX,1670205623
82,82,sur,22.5667,59.5289,22.66,58,0,2.42,OM,1670205648
174,174,dakar,14.6937,-17.4441,23.07,88,0,1.03,SN,1670205600
275,275,mackay,-21.1500,149.2000,26.27,61,0,4.06,AU,1670205740
335,335,doha,25.2867,51.5333,22.99,68,0,1.03,QA,1670205668
343,343,saint-pierre,-21.3393,55.4781,23.28,53,0,3.09,RE,1670205771
384,384,pinotepa nacional,16.3167,-98.0167,22.74,71,0,1.36,MX,1670205790
402,402,acapulco,16.8634,-99.8901,26.90,78,0,3.60,MX,1670205801
418,418,champerico,14.3000,-91.9167,26.64,78,0,1.38,GT,1670205811


### Step 3: Create a new DataFrame called `hotel_df`.

In [73]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = Ideal_city_data_df[['City','Country','Lat','Lng','Humidity']].copy()


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df.insert(5,'Hotel Name',"")


# Display sample data
hotel_df


,City,Country,Lat,Lng,Humidity,Hotel Name
12,bitkine,TD,11.9801,18.2138,15,
36,dzilam gonzalez,MX,21.2833,-88.9333,74,
82,sur,OM,22.5667,59.5289,58,
174,dakar,SN,14.6937,-17.4441,88,
275,mackay,AU,-21.1500,149.2000,61,
335,doha,QA,25.2867,51.5333,68,
343,saint-pierre,RE,-21.3393,55.4781,53,
384,pinotepa nacional,MX,16.3167,-98.0167,71,
402,acapulco,MX,16.8634,-99.8901,78,
418,champerico,GT,14.3000,-91.9167,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [99]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
            "limit": "1",
            "apiKey": g_key
}


# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    # Convert the API response to JSON format
    name_address = requests.get(base_url, params=params).json()
   
  
  
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bitkine - nearest hotel: No hotel found
dzilam gonzalez - nearest hotel: No hotel found
sur - nearest hotel: Sur Hotel
dakar - nearest hotel: 翁记小吃
mackay - nearest hotel: Mackay Oceanside Central Apartment Hotel
doha - nearest hotel: Al MIrqab
saint-pierre - nearest hotel: Tropic Hotel
pinotepa nacional - nearest hotel: No hotel found
acapulco - nearest hotel: Hotel del Valle
champerico - nearest hotel: Hotel y Restaurante El Submarino
pedernales - nearest hotel: Hostal Doña Chava
coahuayana - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
12,bitkine,TD,11.9801,18.2138,15,No hotel found
36,dzilam gonzalez,MX,21.2833,-88.9333,74,No hotel found
82,sur,OM,22.5667,59.5289,58,Sur Hotel
174,dakar,SN,14.6937,-17.4441,88,翁记小吃
275,mackay,AU,-21.1500,149.2000,61,Mackay Oceanside Central Apartment Hotel
335,doha,QA,25.2867,51.5333,68,Al MIrqab
343,saint-pierre,RE,-21.3393,55.4781,53,Tropic Hotel
384,pinotepa nacional,MX,16.3167,-98.0167,71,No hotel found
402,acapulco,MX,16.8634,-99.8901,78,Hotel del Valle
418,champerico,GT,14.3000,-91.9167,78,Hotel y Restaurante El Submarino


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [111]:
%%capture --no-display


# Configure the map plot
mat_plot_map2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",  
    geo = True,
    tiles = 'OSM',
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.8,
    color = "City",
    hover_cols = ["Hotel Name","Country"],
    legend = 'right')



# Display the map
mat_plot_map2



:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)